# Article Review & Reproduction

This notebook is a work due for Pr. Nelly Pustelnik's machine learning class of the M2 complex systems at ENS de Lyon. In the following we review an article introducing physics-informed neural networks and reproduce some of its results.

**Students**: Ayoub Dhibi & Théodore Farmachidi

**Article**:
[Maziar Raissi, , Paris Perdikaris, George Em Karniadakis. "Physics Informed Deep Learning (Part I): Data-driven Solutions of Nonlinear Partial Differential Equations." (2017).](https://arxiv.org/abs/1711.10561)

**Note**: the presentation is expected to include a general introduction, the article's specific features in relation to the state of the art, a more technical section on the article's contributions, and a presentation of the results reproduced.

---

## Table of Contents
1. [Article Summary](#article-summary)
2. [Key Concepts & Methods](#key-concepts--methods)
3. [Important Figures & Results](#important-figures--results)
4. [Reproduction of Results](#reproduction-of-results)
5. [Personal Notes & Questions](#personal-notes--questions)

---

**Roadmap**:

1. Analyze the article and its details
2. Reproduce two examples (one per part)
3. Apply the method to a new use case
4. Prepare the presentation (10' presentation + 10' questions)

**Due dates**: 
- ML challenge for the 04/01
- notebook for the 05/01
- presentation for the 09/01

## 1. Article Explained <a id="article-summary"></a>

In what follows we reformulate the article's main ideas to deepen our understanding and explain technicalities when needed.

### 1. Introduction
When enough data is available neural networks (NNs) have proven very efficient at solving machine learning problems such as natural language processing, image recognition etc. However in many applications the data is scarce and difficult to obtain (complex physical systems, biological systems, etc), with few data, current state-of-the art NNs are less accurate and may not even converge to a proper solution of the problem.

The current approach to solve these complex systems problems do not use all the information available to constraint the solutions. Typically one feeds a neural network (NN) with data pairs of training and target points and trains the NN by minimizing some error criterion with a potential regularization the loss and regularization term are chosen heuristically. When we have a prior knowledge of the system and the physical laws governing it a better approach is to encode directly the physical constraints into the regularization. The resulting model, a physics-informed neural network (PINN) display better convergences guarantees and better generalization even with few data points available.

Previous work on physics-informed models used Gaussian process regression to solve linear problems. They got good approximations and theoretical error boundaries on different linear problems from mathematical physics. Further works extended the method to non linear problems by using the method on local linear approximations of the problem.

#### 1.1 Problem setup and summary of contributions
In the article, they use deep neural networks (DNNs) as universal function approximators extending naturally the class of problems solvable to non linear settings. They also use [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) to compute efficiently the partial derivatives. Then a constraint is added to the neural network so that the approximated function is solution to the non linear partial differential equations (PDEs) ruling the problem these equations encode all the physical laws characteristic of the system (conservation principles, symmetries, etc). This new approach lead to a new class of numerical solvers for PDEs and data-driven model inference.

This new approach can be used to tackle two types of problems,
1. data-driven solution: given noisy measurements find a function that is solution to the PDE 
2. data-driven discovery: given noisy measurements find the parameters of the PDE such that its solution(s) could have produced the measurements
We can frame these as follow, lets consider parametrized non linear partial differential equations of the general form
$$
\partial_t u(t,x) + N[u;\lambda] = 0
$$
where $u(t,x)$ is the solution of the PDE and $N[.;\lambda]$ is a non linear operator parametrized by $\lambda$.
Let us introduce the following notations of subscripts as partial derivatives:
$$
\partial_t u = u_t \text{, } \partial_x u = u_x \text{, } \partial_x^2 u = u_{xx} \text{, etc}
$$
For example Burger's equation
$$
u_t + \lambda_1 uu_x - \lambda_2 uu_x = 0
$$
has $N[u;\lambda] = \lambda_1 uu_x - \lambda_2 uu_x$ with $\lambda = (\lambda_1, \lambda_2)$.
The two types of problems we outlined can now be reframed as:
1. data-driven solution: given noisy measurements and fixed parameters $\lambda$ what can be said about $u$
2. data-driven discovery: given noisy measurements and assuming they come from a solution of the equation, what are the most likely parameters $\lambda$ ruling the equation

This paper focuses on data-driven solutions to the restricted class of problems:
$$
u_t + N[u] = 0, \ x \in \Omega, \ t \in [0,T]
$$
with $\Omega$ a subset of $\mathbb{R}^D$. The paper presents two classes of algorithms, continuous and discrete time models and evaluate their properties and performance on benchmark problems. For reproducibility the author made the code available at [https://github.com/maziarraissi/PINNs](https://github.com/maziarraissi/PINNs).


### 2. Continuous Time Models
A deep neural network is used to approximate a solution of the PDE. If the network is deep enough it is an universal approximator i.e. it can approximate any function one will ever encounter (except very pathological examples) [ref](https://www.sciencedirect.com/science/article/pii/0893608089900208). We denote the neural network as $u^\theta$ where $\theta$ represents the parameters of the neural network. Finally we define the *physics informed neural network*
$$
f = u_t^\theta + N[u^\theta]
$$
Which can be derived via automatic differentiation.<span style="color:red"> Automatic differentiation</span> is a way to compute exact derivatives of numerical functions that are expressed as computational graphs e.g. neural networks.

The neural networks $u^\theta$ and $f$ are trained by minimizing
$$
MSE = MSE_u + MSE_f
$$

where

$$
MSE_u = \frac{1}{N_u} \sum_{i=1}^{N_u} |u(t_u^i, x_u^i) - u^i|^2,
$$

and

$$
MSE_f = \frac{1}{N_f} \sum_{i=1}^{N_f} |f(t_f^i, x_f^i)|^2.
$$

With $MSE_u$ computed over boundary and limit conditions training data for $u$ and $MSE_f$ computed on collocation points sampled randomly inside the support $\Omega$ via <span style="color:red"> Latin Hypercube Sampling Strategy </span>.

If the PDE problem is well posed, the boundary and limit conditions along are enough to determine an unique solution. Moreover, typical PDE solvers (based on other methods e.g. finite differences) also use only boundary and limit data. However there is no theoretical convergence guarantee. But the empirical results shows that with a deep enough neural network and with enough collocation points $N_f$, the approximated solution achieves good prediction accuracy.

In what follows the authors illustrate the continuous time method on Burger's equation and on Schrodinger's equation. They try different set of parameters to build an empirical result on the model accuracy as mentioned above.

We want enough collocation points to avoid over-fitting. Big $N_f$ = strong regularization.

#### 2.1 Burger's equation
##### a. Problem definition
Burger's equation can be derived from the Navier-Stokes equations for the velocity field by dropping the pressure gradient term. Burger's equation are used in fluid mechanics, non linear acoustics, gas dynamics and traffic flow. In 1D the Burger's equation writes,
$$
u_t + uu_x - (0.01/\pi)u_{xx} = 0, \ \forall x \in [-1,1], \ \forall t \in [0,1]
$$
**If we add** the Dirichlet boundary conditions,
$$
u(0,x) = -\sin (\pi x) \\
u(t,1) = u(t,1) = 0
$$
the Burger's equation as stated has an unique solution. We now look for a neural network $u^\theta$ that will approximate this solution. We define the physics informed neural network $f$ as we explained it above,
$$
f = u^{\theta}_t + u^\theta u_{x}^\theta - (0.01/\pi)u_{xx}^\theta
$$

##### b. Solver setup

- **Network Architecture** : The neural network $u^\theta$ has 9 layers of 20 neurons ($= 3021$ parameters) with $\tanh$ activation functions.

<center>


| Input layer | Hidden layers | Output layer |
|-------------|---------------|--------------|
|[2] = $(x,t)$|    [20] x 8 w/ $\tanh$  |[1] = $u(x,t)$|

</center>

<details>
<summary>Note</summary>

---
Here the paper can be confusing because it says
> "the network architecture is fixed to 9 layers with 20 neurons per hidden layer"

It seems that either the input or output layer is not considered as a layer by the author. However the architecture above is the exact one used cf. [line 145 of the python script](https://github.com/maziarraissi/PINNs/blob/master/appendix/continuous_time_inference%20(Burgers)/Burgers.py) used to generate the results in the article.

---
</details>



- **Optimizer**: Loss functions were optimized using <span style="color:red">L-BFGS</span> algorithm.
For bigger datasets, the authors suggest to use a mini-batch setting with SGD/modern variants.

- **Training data**: 
    - BC/IC points: In the python [script](https://github.com/maziarraissi/PINNs/blob/master/appendix/continuous_time_inference%20(Burgers)/Burgers.py), authors load a file with the exact solution values computed on a regular grid. Create a vector with all the points on the boundary and at initial time $t=0$. Sample uniformly $N_u=100$ points from this vector to make the initial training set mimicking sparse experimental sampling.
    - Collocation points: $N_f$ points sampled using <span style="color:red">Latin Hypercube Strategy</span>


- **Methodology**:
    1. With the fixed neuron architecture (9 layers) the authors computed the $\mathcal{L}_2$ error between the predicted and the exact solution for varying $N_u$ (20->200) and $N_f$ (2000->10 000).
    2. Then for fixed $(N_u, N_f)=(100, 10\ 000)$ the authors computed the $\mathcal{L}_2$ error between the predicted and the exact solution for varying number of layers (2->8) and neurons per layer (10->40).

- **Results**:
    1. Relative error $\mathcal{L}_2$ when both $N_u$ and $N_f$ increase. Authors emphasize that even with a small amount of boundary and limit points, the neural network yields good prediction results thanks to the tunable number of "virtual" collocation points that compensate for the lack of data.
    2. As expected, $\mathcal{L}_2$ error decreases with the number of layers and neurons per layer. The interpretation is that the neural network gets more expressive i.e. as the number of parameters increases it gets better at approximating complex function. 


#### 2.2 Shrodinger equation
Authors chose to demonstrate their method on another example, the *Shrodinger Equation*, to display the versatility of PINNs.
Novelties compared to Burger's equation and how they are tackled:

- complex values : multi-output neural network outputs = $[u(t,x) \ v(t,x)]$ where $u$ and $v$ are the real and imaginary parts of the solution to the *Schrodinger Equation*

- periodic boundaries conditions : added a regularization term

- different nonlinearities: more expressive neural network

- **Network Architecture**: They use a 5 layers 100 neurons network that is more expressive than the Burger's one. This network have $30\ 802$ parameters.
<center>


| Input layer | Hidden layers | Output layer |
|-------------|---------------|--------------|
|[2] = $(x,t)$|    [100] x 4 w/ $\tanh$    |[2] = $\left[u(t,x) \ v(t,x)\right]$|

</center>

- **Methodology**:
    1. Generate an high resolution solution to the PDE with already existing integrators in python. 
    2. Sample at random 50 points for the initial condition and 50 points on the boundaries. LHS sampling of 20 000 collocation points inside the solution domain.
    3. Measure $\mathcal{L}_2$ error between the predicted and exact (from the solver) solution.

- **Results**: $\mathcal{L}_2$ error $= 1.97.19^{-3}$, which is low but still one order of magnitude greater than the results obtained with the best network for Burger's equation. However no systematic was done here.

#### Continuous time models limitations
The authors highlight:
- need for a large number of collocation points to enforce the physics $\rightarrow$ increases exponentially with dimension

Solution = next section = discrete time models 

### 3. Discrete Time Models

Given a PDE and boundary conditions that we assume have an unique solution, we have data from the solution at time $t^n$ and over all the $x$ domain. What does the solution looks like at a further time $t^{n+1} := t^n +\Delta t$.

We have the PDE,
$$
u_t + N[u] = 0, \ x \in \Omega, \ t \in [0,T]
$$

For a fixed $x \in \Omega$, this equation is an ODE with respect to time,
$$
\frac{d}{dt}u(t) + N[u(t)] = 0, \forall t \in [0,T]
$$

**The Discrete Time Model Explained**:
1. The $q$ stages Runge-Kutta scheme gives us $q+1$ equations that must be satisfied by a function that correctly approximates the solution at time $t^{n+1}$.
    $$
    \begin{align*}
        u^{n+c_i} &= u^n - \Delta t \sum_{j=1}^q a_{ij} \mathcal{N}\left[u^{n+c_j}\right], \quad i = 1, \ldots, q, \\
        u^{n+1} &= u^n - \Delta t \sum_{j=1}^q b_j \mathcal{N}\left[u^{n+c_j}\right].
    \end{align*}
    $$
    The coefficients $a, b \text{ and } c$ are given by the type of RK scheme, it can be explicit or implicit.


2. These equations can be rewritten as,
    $$
    \begin{align*}
        u^n &= u^n_i, \quad i = 1, \ldots, q, \\
        u^n &= u^n_{q+1}, \tag{$\ast$}
    \end{align*}
    $$

    Where,
    $$
    \begin{align*}
        u^n_i &:= u^{n+c_i} + \Delta t \sum_{j=1}^q a_{ij} \mathcal{N}\left[u^{n+c_j}\right], \quad i = 1, \ldots, q, \\
        u^n_{q+1} &:= u^{n+1} + \Delta t \sum_{j=1}^q b_j \mathcal{N}\left[u^{n+c_j}\right].
    \end{align*}
    $$


3. We train a neural network that takes $x$ in input and outputs $[u^{n+c_1}(x), \ldots, u^{n+c_q}(x), u^{n+1}(x)]^T$ then using the RK formulas and automatic differentiation we compute $[u^n_1(x),\ldots,u^n_q(x), u^n_{q+1}(x)]^T$.

    **How do we train the network ?**

    The network 1D input is fed an $x_k$ point from the training data. The target is $u^n(x_k)$ more exactly it is a $q+1$ dimensional vector $[u^n(x_k),\ldots, u^n(x_k)]^T$ because from $(\ast)$, we should have $$[u^n(x_k),\ldots, u^n(x_k)]^T = [u^n_1(x_k),\ldots,u^n_q(x_k), u^n_{q+1}(x_k)]^T \ \ \ \forall k $$
    A condition to fit the boundary conditions is also added to the final loss.

    Once the network is trained we have a neural network specific to the time step $t^n \rightarrow t^{n+1}$ and this $\forall x$ in the domain.













**Definitons**:
- **collocation points** = randomly chosen points on the support where the neural network is forced to satisfy the PDE.

**Notes**:
- In the pdf at page 4 the author explain the two types of problems and mentions "filtering and smoothing" for data driven solutions. He also makes extensive citations to previous work he conducted, maybe I need to also reed his other papers to get a better solutions of these "data-driven" problems.
- Text in <span style="color:red">red</span> refers to technical details that we have to explain in more depth.
- Apparently for these problems the L-BFGS algorithm is really better suited, we can try to compare it with Adam optimizer.
- Compare computational time between the discrete and continuous time methods

## 2. Key Concepts and Methods
---
### L-BFGS algorithm
L-BFGS is 
- a quasi-Newton, full-batch gradient-based optimization algorithm 

**gradient-based optimization**: 
To find the minimum of a loss function $\mathcal{L}$ the algorithm incrementally goes down the steepest direction of the $\mathcal{L}$. Locally this direction is given by the larger in absolute value negative gradient with respect to the parameters $\nabla\mathcal{L}(\theta)$.

$$
\theta_{k+1} = \theta_k - \mathbf{H}^{-1}\nabla\mathcal{L}
$$

**quasi-Newton**: 
Newton methods update automatically the learning rate of gradient descent algorithms to make big steps in flat areas and small steps in steep areas. To do so they use the local curvature of the loss function $\mathcal{L}$ during the optimization. The local curvature is encoded by the Hessian $\mathbf{H}$ so the updated parameters will take a similar form as,
$$
\theta_{k+1} = \theta_k - \mathbf{H}^{-1}\nabla\mathcal{L}
$$
However the Hessian big and therefore it takes space to store it and time to invert it so quasi-Newton methods build and approximation of $\mathbf{H}^{-1}$

**full-batch**: 
Every gradient is computed using **all** collocation and boundary/limit points, no stochasticity is used.



---
### Latin Hypercube Sampling (LHS) strategy
LHS is a sampling strategy that enforces coverage of the sampled domain while keeping the sampling random/asymmetric.

**How does it work ?**

In the article we want to sample $N_f$ points in a 2D domain $(x,t) = [-1,1] \times [0,1]$. 
1. The algorithm divides each axis into $N_f$ equal-probability bins
2. Picks **one** point in each bin on each axis
3. Create $N_f$ random pairs of $(x,t)$ bins

This results in **one** sample in each row and each column

**Why not using uniform sampling instead ?**

Uniform sampling could result in random inhomogeneous sampling density over the integration support therefore impacting negatively the accuracy of the model.

---

### Runge-Kutta Scheme


## 3. Important Figures & Results <a id="important-figures--results"></a>

_Add screenshots, plots, or descriptions of the most important figures and results from the article. Briefly explain their significance._

## 4. Reproduction of Results <a id="reproduction-of-results"></a>

As the authors provide the code to reproduce their results, we reproduce the exact results in the paper and then go further by applying the continuous and discrete methods to a new PDE problem.

## 5. Personal Notes & Questions <a id="personal-notes--questions"></a>

_Write down your thoughts, questions, and points to discuss during your presentation or with your professor._